Using Little VGG for emotion detection

Training Emotion Detector

In [7]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization,Conv2D,MaxPooling2D
import os

num_classes=6
img_rows,img_cols=48,48
batch_size=18

train_data_dir='./fer2013/train'
validation_data_dir='./fer2013/validation'

#data augmentation
train_datagen=ImageDataGenerator(
             rescale=1./255,
             rotation_range=30,
             shear_range=0.5,
             width_shift_range=0.6,
             height_shift_range=0.3,
             horizontal_flip=True,
             fill_mode='nearest')

validation_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
            train_data_dir,
            color_mode='grayscale',
            target_size=(img_rows,img_cols),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=True)

validation_generator=validation_datagen.flow_from_directory(
            validation_data_dir,
            color_mode='grayscale',
            target_size=(img_rows,img_cols),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=True)

Found 28273 images belonging to 6 classes.
Found 3534 images belonging to 6 classes.


Keras Little VGG Model

In [8]:
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.core import Activation,Flatten,Dropout,Dense

In [9]:
model=Sequential()

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(512,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(512,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

In [10]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint=ModelCheckpoint('./emotion_little_vgg.h5',
                          monitor='val_loss',
                          mode='min',
                          save_best_only=True,
                          verbode=1)

earlystop=EarlyStopping(monitor='val_loss',
                       min_delta=0,
                       patience=3,
                       verbose=1,
                       restore_best_weights=True)

reduce_lr=ReduceLROnPlateau(monitor='val_loss',
                            factor=0.2,
                            patience=3,
                            verbose=1,min_delta=0.0001)

callbacks=(earlystop,checkpoint,reduce_lr)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

nb_train_samples=28273
nb_validation_samples=3534
epochs=10

history=model.fit_generator(
train_generator,
steps_per_epoch=nb_train_samples//batch_size,
epochs=epochs,
callbacks=callbacks,validation_steps=nb_validation_samples//batch_size,validation_data=validation_generator)

c:\python39\lib\site-packages\keras\engine\training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1570/1570 [==============================] - 463s 286ms/step - loss: 2.2547 - accuracy: 0.1967 - val_loss: 1.7447 - val_accuracy: 0.2523
Epoch 2/10
1570/1570 [==============================] - 466s 297ms/step - loss: 1.7563 - accuracy: 0.2444 - val_loss: 1.7403 - val_accuracy: 0.2531
Epoch 3/10
1570/1570 [==============================] - 447s 285ms/step - loss: 1.7308 - accuracy: 0.2618 - val_loss: 1.6886 - val_accuracy: 0.2931
Epoch 4/10
1570/1570 [==============================] - 464s 296ms/step - loss: 1.6861 - accuracy: 0.2932 - val_loss: 1.5846 - val_accuracy: 0.3509
Epoch 5/10
1570/1570 [==============================] - 437s 278ms/step - loss: 1.6080 - accuracy: 0.3372 - val_loss: 1.6277 - val_accuracy: 0.4093
Epoch 6/10
1570/1570 [==============================] - 448s 285ms/step - loss: 1.5107 - accuracy: 0.3908 - val_loss: 1.4170 - val_accuracy: 0.4374
Epoch 7/10
1570/1570 [==============================] - 437s 279ms/step - loss: 1.4655 - accuracy: 0.4146 - val_